#### import all required library

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium
import json
import requests
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
import seaborn as sns

#### Read the csv, drop unwanted columns and data

In [2]:
df=pd.read_csv('India postoffice.csv') #Upload Data
df.drop(['officeType','Deliverystatus','divisionname','circlename','Taluk','Districtname',
         'statename','Telephone','Related Suboffice','Related Headoffice','longitude','latitude'],axis=1,inplace=True) # drop unwanted columns
df.drop_duplicates(subset=['pincode'],inplace=True) # drop duplicate records
dfmumbai=df[(df['pincode']>400000) & (df['pincode']<401000)] # make new dataframe for mumbai with pincode between 400000 and 401000
dfmumbai.head()

,officename,pincode,regionname
81395,Antop Hill S.O,400037,Mumbai
81397,B.P.Lane S.O,400003,Mumbai
81398,BEST STaff Quarters S.O,400012,Mumbai
81401,Chinchbunder H.O,400009,Mumbai
81402,Cotton Exchange S.O,400033,Mumbai


In [3]:
dfpune=df[(df['pincode']>411000) & (df['pincode']<411100)] # make new dataframe for pune with pincode between 411000 and 411100
dfpune.head()

,officename,pincode,regionname
86084,9 DRD B.O,411014,Pune
86085,AFMC S.O,411040,Pune
86086,Airport S.O (Pune),411032,Pune
86087,Akurdi S.O,411035,Pune
86088,Ammunition Factory Khadki S.O,411003,Pune


In [4]:
dfnagpur=df[(df['pincode']>440000) & (df['pincode']<440100)]# make new dataframe for nagpur with pincode between 440000 and 440100
dfnagpur.head()

,officename,pincode,regionname
84241,Abhyankar Nagar S.O,440010,Nagpur
84242,Ajni S.O,440003,Nagpur
84243,Ayodhya Nagar S.O,440024,Nagpur
84244,Bagadganj S.O,440008,Nagpur
84245,BESA Road S.O,440037,Nagpur


In [5]:
def findlatlng(dfcity): # find latitude and longitude of pin codes and update the dataframe
    geolocator=Nominatim(user_agent="India_explorer")
    for addr in dfcity['pincode']:
        addr1=str(addr)+",India" 
        location=geolocator.geocode(addr1) # Check for location
        if location!=None: # if location found update dataframe with latitude and longitude
            dfcity.loc[dfcity['pincode']==addr,'latitude']=location.latitude
            dfcity.loc[dfcity['pincode']==addr,'longitude']=location.longitude
        else: # if not found update dataframe with NAN
            dfcity.loc[dfcity['pincode']==addr,'latitude']=np.nan
            dfcity.loc[dfcity['pincode']==addr,'longitude']=np.nan
    dfcity.dropna(inplace=True)    
    return dfcity

In [6]:
dfmumbai=findlatlng(dfmumbai)# find latitude and longitude of Mumbai pin codes and update the dataframe
dfmumbai.head()

,officename,pincode,regionname,latitude,longitude
0,Antop Hill S.O,400037,Mumbai,19.023074,72.867622
1,B.P.Lane S.O,400003,Mumbai,18.951606,72.834797
2,BEST STaff Quarters S.O,400012,Mumbai,19.000795,72.840147
3,Chinchbunder H.O,400009,Mumbai,18.959645,72.838526
4,Cotton Exchange S.O,400033,Mumbai,18.981780,72.840388


In [7]:
dfpune=findlatlng(dfpune) # find latitude and longitude of Pune pin codes and update the dataframe
dfpune.head()

,officename,pincode,regionname,latitude,longitude
0,9 DRD B.O,411014,Pune,18.559096,73.920485
1,AFMC S.O,411040,Pune,18.484045,73.901684
2,Airport S.O (Pune),411032,Pune,18.583178,73.900781
3,Akurdi S.O,411035,Pune,18.771235,73.966942
4,Ammunition Factory Khadki S.O,411003,Pune,18.571007,73.838327


In [8]:
dfnagpur=findlatlng(dfnagpur) # find latitude and longitude of Nagpur pin codes and update the dataframe
dfnagpur.head()

,officename,pincode,regionname,latitude,longitude
0,Abhyankar Nagar S.O,440010,Nagpur,21.133355,79.068431
1,Ajni S.O,440003,Nagpur,21.124036,79.102410
2,Ayodhya Nagar S.O,440024,Nagpur,21.120171,79.117492
3,Bagadganj S.O,440008,Nagpur,21.145063,79.129437
4,BESA Road S.O,440037,Nagpur,21.080441,79.077285


In [9]:
def showmap(Loc,dfcity): #Function to create map with City name as location and dataframe with different locality
    geolocator=Nominatim(user_agent="India_explorer")
    location=geolocator.geocode(Loc) # find location of the city
    map=folium.Map(location=[location.latitude,location.longitude],zoom_start=11) # create map of city with zoom off 11
    for lat,lng,label in zip(dfcity['latitude'],dfcity['longitude'],dfcity['officename']): 
        label=folium.Popup(label,parse_html=True)
        folium.CircleMarker(
            [lat,lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='blue',
            fill_opacity=0.7,
            parse_html=False).add_to(map) # update all neighborhood with blue circle
    return map

In [10]:
mumbaimap=showmap('Mumbai',dfmumbai)# create map of Mumbai with all neighborhood
mumbaimap

In [11]:
punemap=showmap('Pune',dfpune) # create map of Pune with all neighborhood
punemap

In [12]:
nagpurmap=showmap('Nagpur',dfnagpur) # create map of Nagpur with all neighborhood
nagpurmap

#### Use Foursquare API to find nearby Location

In [13]:
def getNearbyVenues(names,latitudes,longitudes,radius=500): # Function to find Nearby Venues in 500 mm radius 
    venues_list=[]
    for name,lat,lng in zip(names,latitudes,longitudes):
        url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)# Create URL
        results = requests.get(url).json()["response"]['groups'][0]['items'] #get details from the API
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results]) # update result in list

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list]) # Convrt list to dataframe
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category'] # update column names to dataframe
    
    return(nearby_venues)

In [14]:
venuesmumbai = getNearbyVenues(names=dfmumbai['officename'],
                                   latitudes=dfmumbai['latitude'],
                                   longitudes=dfmumbai['longitude']
                                  ) # Update nearby Venues for each neighborhood of Mumbai
venuesmumbai.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Antop Hill S.O,19.023074,72.867622,Polygomma Industries Pvt. Ltd.,19.023255,72.867132,Construction & Landscaping
1,Antop Hill S.O,19.023074,72.867622,naaz supermarket,19.022164,72.863284,Grocery Store
2,Antop Hill S.O,19.023074,72.867622,Palkhi Restaurant,19.022163,72.863034,Indian Restaurant
3,B.P.Lane S.O,18.951606,72.834797,Gulshan-E-Iran,18.948118,72.835427,Middle Eastern Restaurant
4,B.P.Lane S.O,18.951606,72.834797,Bhagat Tarachand Restaurant,18.951802,72.830486,Indian Restaurant


In [15]:
venuespune = getNearbyVenues(names=dfpune['officename'],
                                   latitudes=dfpune['latitude'],
                                   longitudes=dfpune['longitude']
                                  ) # Update nearby Venues for each neighborhood of Pune
venuespune.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,9 DRD B.O,18.559096,73.920485,95 Pasta N Pizza,18.561009,73.920031,Italian Restaurant
1,9 DRD B.O,18.559096,73.920485,Costa Coffee,18.560990,73.920420,Coffee Shop
2,9 DRD B.O,18.559096,73.920485,PVR bluO,18.561614,73.917133,Bowling Alley
3,9 DRD B.O,18.559096,73.920485,Phoenix Market City,18.561951,73.916895,Shopping Mall
4,9 DRD B.O,18.559096,73.920485,Shizusan,18.561722,73.916775,Asian Restaurant


In [16]:
venuesnagpur = getNearbyVenues(names=dfnagpur['officename'],
                                   latitudes=dfnagpur['latitude'],
                                   longitudes=dfnagpur['longitude']
                                  ) # Update nearby Venues for each neighborhood of Nagpur
venuesnagpur.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Abhyankar Nagar S.O,21.133355,79.068431,Hotel Tuli Imperial,21.133568,79.073011,Hotel
1,Abhyankar Nagar S.O,21.133355,79.068431,Vishnuji Kee Rasoi,21.131228,79.065099,Indian Restaurant
2,Abhyankar Nagar S.O,21.133355,79.068431,Law College Square,21.133103,79.069915,Outdoors & Recreation
3,Abhyankar Nagar S.O,21.133355,79.068431,Nagpur Central Mall,21.137328,79.068703,Shopping Mall
4,Abhyankar Nagar S.O,21.133355,79.068431,Bonsaii,21.135261,79.068911,Gift Shop


In [17]:
def onehot(venues): #Function to create onehot of a column in DataFrame
    venue_onehot=[]
    venue_onehot=pd.get_dummies(venues[['Venue Category']],prefix="",prefix_sep="") # create onehot
    venue_onehot['Neigh']=venues['Neighborhood']# Add Neighborhood column
    fixed_columns = [venue_onehot.columns[-1]] + list(venue_onehot.columns[:-1])
    venue_onehot = venue_onehot[fixed_columns]# Make Neighborhood column the first column
    return venue_onehot

In [18]:
def groupvenue(venue_onehot): # Function to return grouped dataframe
    venue_grouped=venue_onehot.groupby('Neigh').mean().reset_index() # group the dataframe on Neighborhood column
    return venue_grouped

In [19]:
def return_most_common_venues(row, num_top_venues): # Function to return top common venues
    row_categories = row.iloc[1:] # shift venues to a list
    row_categories_sorted = row_categories.sort_values(ascending=False) # sort the list with most common on the top
    return row_categories_sorted.index.values[0:num_top_venues] # return the list with most common venues as per user 

In [20]:
def top10venues(venue_grouped): # Function to create a Dataframe showing top 10 venues for each neighborhood
    num_top_venues = 10

    indicators = ['st', 'nd', 'rd']

    # create columns according to number of top venues
    columns = ['Neighborhood']
    for ind in np.arange(num_top_venues):
        try:
            columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
        except:
            columns.append('{}th Most Common Venue'.format(ind+1))

    # create a new dataframe
    venues_sorted = pd.DataFrame(columns=columns)# create dataframe with the above column
    venues_sorted['Neighborhood'] = venue_grouped['Neigh']#update Neighborhhod column
    # Update all the Neighborhood with top 10 venues
    for ind in np.arange(venue_grouped.shape[0]):
        venues_sorted.iloc[ind, 1:] = return_most_common_venues(venue_grouped.iloc[ind, :], num_top_venues)

    return venues_sorted

In [21]:
def Cal_kmeans(venue_grouped): #Function to calculate kmeans
    k=5
    venue_cluster=venue_grouped.drop('Neigh',axis=1) # Drop the Neighborhood Column as Kmeans requires all numeric values
    kmean=KMeans(n_clusters=k,random_state=0).fit(venue_cluster) # Calculate kmeans with 5 clusters
    return kmean

In [22]:
def mergedf(dfcity,venues_sorted): # Function to merge Kmean and top 10 venues database
    dfcity['Neighborhood']=dfcity['officename']# change colmumn name
    venues_merged=dfcity.drop(['officename'],axis=1)# drop unwanted column
    venues_merged=venues_merged.join(venues_sorted.set_index('Neighborhood'),on='Neighborhood') # merge the dataframe on Neighborhood column
    venues_merged.drop(venues_merged[venues_merged['Cluster Labels'].isnull()].index,inplace=True)# remove null column
    venues_merged['Cluster Labels']=venues_merged['Cluster Labels'].astype(int)# Convert clustr Labels data to int
    return venues_merged

In [23]:
def showclustermap(Loc,venues_merged,k=5): # Function to generate map showing clusters in different color
    geolocator=Nominatim(user_agent="India_explorer")
    location=geolocator.geocode(Loc) # find location of city
    map = folium.Map(location=[location.latitude,location.longitude], zoom_start=11) # Create Map of city with zoom of 11

    # set color scheme for the clusters
    x = np.arange(k)
    ys = [i + x + (i*x)**2 for i in range(k)]
    colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
    rainbow = [colors.rgb2hex(i) for i in colors_array]

    # add  cluster markers to the map
    markers_colors = []
    for lat, lon, poi, cluster in zip(venues_merged['latitude'], venues_merged['longitude'], venues_merged['Neighborhood'], venues_merged['Cluster Labels']):
        label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[cluster-1],
            fill=True,
            fill_color=rainbow[cluster-1],
            fill_opacity=0.7).add_to(map)
    return map

In [24]:
len(venuesmumbai['Venue Category'].unique()) # Count Unique Venues for Mumbai

184

In [25]:
len(venuespune['Venue Category'].unique()) # Count Unique Venues for Pune

97

In [26]:
len(venuesnagpur['Venue Category'].unique()) # Count Unique Venues for Nagpur

45

In [27]:
# Create 10 Most common venues for each neighborhood in Mumbai
mumbai_onehot=onehot(venuesmumbai)
mumbai_grouped=groupvenue(mumbai_onehot)
mumbai_sorted=top10venues(mumbai_grouped)
mumbai_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,A I Staff Colony S.O,Indian Restaurant,Smoke Shop,Chinese Restaurant,Lounge,Fast Food Restaurant,Sporting Goods Shop,Bar,Tea Room,Food Truck,Pizza Place
1,Agripada S.O,Trail,Diner,Bar,Zoo,Farm,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market
2,Airoli B.O,Business Service,Zoo,Farmers Market,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market,Field
3,Airport S.O (Mumbai),Hotel,Café,Gym,Farm,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market,Field
4,Ambewadi S.O (Mumbai),Indian Restaurant,Dessert Shop,Fried Chicken Joint,Ice Cream Shop,Antique Shop,BBQ Joint,Restaurant,Farmers Market,Food & Drink Shop,Food


In [28]:
# Create 10 Most common venues for each neighborhood in Pune
pune_onehot=onehot(venuespune)
pune_grouped=groupvenue(pune_onehot)
pune_sorted=top10venues(pune_grouped)
pune_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,9 DRD B.O,Café,Fast Food Restaurant,Hotel,Pizza Place,Indian Restaurant,Italian Restaurant,Diner,Sandwich Place,Hookah Bar,Coffee Shop
1,A.R. Shala S.O,Indian Restaurant,Snack Place,Seafood Restaurant,Fast Food Restaurant,Gourmet Shop,Ice Cream Shop,Brewery,Bus Station,Café,Chinese Restaurant
2,AFMC S.O,Café,Italian Restaurant,Wine Shop,Falafel Restaurant,Coffee Shop,Liquor Store,Chinese Restaurant,Other Great Outdoors,Diner,Donut Shop
3,Airport S.O (Pune),Recreation Center,Lake,Wine Shop,Dumpling Restaurant,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Distillery
4,Ambegaon BK B.O,ATM,Department Store,Bus Station,Farm,Electronics Store,Dessert Shop,Dim Sum Restaurant,Diner,Distillery,Donut Shop


In [29]:
# Create 10 Most common venues for each neighborhood in Nagpur
nagpur_onehot=onehot(venuesnagpur)
nagpur_grouped=groupvenue(nagpur_onehot)
nagpur_sorted=top10venues(nagpur_grouped)
nagpur_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,A.D. Project S.O,Coffee Shop,Ice Cream Shop,Lake,Dessert Shop,Department Store,Garden,Food Truck,Food Court,Fast Food Restaurant,Dumpling Restaurant
1,Abhyankar Nagar S.O,Gift Shop,Bakery,Gym / Fitness Center,Shopping Mall,Hotel,Park,Outdoors & Recreation,Indian Restaurant,Dessert Shop,Food Truck
2,Adegaon B.O,Mobile Phone Shop,Wings Joint,Gift Shop,Garden,Food Truck,Food Court,Fast Food Restaurant,Dumpling Restaurant,Dhaba,Dessert Shop
3,Ajni S.O,Pool,IT Services,Business Service,Chinese Restaurant,Wings Joint,Department Store,Garden,Food Truck,Food Court,Fast Food Restaurant
4,Ayodhya Nagar S.O,Garden,Food Court,Wings Joint,Gym / Fitness Center,Gift Shop,Food Truck,Fast Food Restaurant,Dumpling Restaurant,Dhaba,Dessert Shop


In [30]:
#Calculate k means and merge the dataframe of Mumbai
mumbai_kmeans=Cal_kmeans(mumbai_grouped)
mumbai_sorted.insert(0,'Cluster Labels',mumbai_kmeans.labels_)
mumbai_merged=mergedf(dfmumbai,mumbai_sorted)
mumbai_merged.head()

,pincode,regionname,latitude,longitude,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,400037,Mumbai,19.023074,72.867622,Antop Hill S.O,0,Construction & Landscaping,Indian Restaurant,Grocery Store,Zoo,Farm,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market
1,400003,Mumbai,18.951606,72.834797,B.P.Lane S.O,0,Indian Restaurant,Convenience Store,Flea Market,Middle Eastern Restaurant,Dessert Shop,American Restaurant,Ice Cream Shop,Restaurant,Indian Sweet Shop,Chinese Restaurant
2,400012,Mumbai,19.000795,72.840147,BEST STaff Quarters S.O,1,Indian Restaurant,Gym / Fitness Center,Bar,Park,Multicuisine Indian Restaurant,Pharmacy,Coffee Shop,Playground,Recording Studio,Restaurant
3,400009,Mumbai,18.959645,72.838526,Chinchbunder H.O,1,Mini Golf,Indian Restaurant,Harbor / Marina,Furniture / Home Store,Café,Brewery,Fast Food Restaurant,Food Court,Food & Drink Shop,Food
4,400033,Mumbai,18.981780,72.840388,Cotton Exchange S.O,1,Pizza Place,Hookah Bar,Furniture / Home Store,Arts & Crafts Store,Falafel Restaurant,Food,Flower Shop,Flea Market,Fish Market,Field


In [31]:
#Calculate k means and merge the dataframe of Pune
pune_kmeans=Cal_kmeans(pune_grouped)
pune_sorted.insert(0,'Cluster Labels',pune_kmeans.labels_)
pune_merged=mergedf(dfpune,pune_sorted)
pune_merged.head()

,pincode,regionname,latitude,longitude,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,411014,Pune,18.559096,73.920485,9 DRD B.O,1,Café,Fast Food Restaurant,Hotel,Pizza Place,Indian Restaurant,Italian Restaurant,Diner,Sandwich Place,Hookah Bar,Coffee Shop
1,411040,Pune,18.484045,73.901684,AFMC S.O,1,Café,Italian Restaurant,Wine Shop,Falafel Restaurant,Coffee Shop,Liquor Store,Chinese Restaurant,Other Great Outdoors,Diner,Donut Shop
2,411032,Pune,18.583178,73.900781,Airport S.O (Pune),2,Recreation Center,Lake,Wine Shop,Dumpling Restaurant,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Distillery
4,411003,Pune,18.571007,73.838327,Ammunition Factory Khadki S.O,0,Chinese Restaurant,Indian Restaurant,Wine Shop,English Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Distillery,Donut Shop
5,411027,Pune,18.582607,73.805725,Aundh Camp S.O,4,Café,Wine Shop,Dance Studio,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Distillery,Donut Shop,Dosa Place


In [32]:
#Calculate k means and merge the dataframe of Nagpur
nagpur_kmeans=Cal_kmeans(nagpur_grouped)
nagpur_sorted.insert(0,'Cluster Labels',nagpur_kmeans.labels_)
nagpur_merged=mergedf(dfnagpur,nagpur_sorted)
nagpur_merged.head()

,pincode,regionname,latitude,longitude,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,440010,Nagpur,21.133355,79.068431,Abhyankar Nagar S.O,2,Gift Shop,Bakery,Gym / Fitness Center,Shopping Mall,Hotel,Park,Outdoors & Recreation,Indian Restaurant,Dessert Shop,Food Truck
1,440003,Nagpur,21.124036,79.102410,Ajni S.O,2,Pool,IT Services,Business Service,Chinese Restaurant,Wings Joint,Department Store,Garden,Food Truck,Food Court,Fast Food Restaurant
2,440024,Nagpur,21.120171,79.117492,Ayodhya Nagar S.O,2,Garden,Food Court,Wings Joint,Gym / Fitness Center,Gift Shop,Food Truck,Fast Food Restaurant,Dumpling Restaurant,Dhaba,Dessert Shop
3,440008,Nagpur,21.145063,79.129437,Bagadganj S.O,2,ATM,Clothing Store,Department Store,Gift Shop,Garden,Food Truck,Food Court,Fast Food Restaurant,Dumpling Restaurant,Dhaba
5,440014,Nagpur,21.166664,79.086990,Bezonbagh S.O,2,Mobile Phone Shop,Bakery,History Museum,Playground,Coffee Shop,Garden,Food Truck,Food Court,Fast Food Restaurant,Dumpling Restaurant


In [33]:
#Create clustered map of Mumbai
mumbai_clustermap=showclustermap('Mumbai',mumbai_merged)
mumbai_clustermap

In [34]:
#Create clustered map of Pune
pune_clustermap=showclustermap('Pune',pune_merged)
pune_clustermap

In [35]:
#Create clustered map of Nagpur
nagpur_clustermap=showclustermap('Nagpur',nagpur_merged)
nagpur_clustermap

#### Explore each Cluster

In [36]:
# Check the neighborhood in Cluster 0 of Mumbai
mumbai_merged.loc[mumbai_merged['Cluster Labels'] == 0, mumbai_merged.columns[[4] + list(range(6, mumbai_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Antop Hill S.O,Construction & Landscaping,Indian Restaurant,Grocery Store,Zoo,Farm,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market
1,B.P.Lane S.O,Indian Restaurant,Convenience Store,Flea Market,Middle Eastern Restaurant,Dessert Shop,American Restaurant,Ice Cream Shop,Restaurant,Indian Sweet Shop,Chinese Restaurant
25,Santacruz(East) S.O,Indian Restaurant,Hotel,Ice Cream Shop,Outdoors & Recreation,Asian Restaurant,Farmers Market,Food Court,Food & Drink Shop,Food,Flower Shop
30,Best Staff Colony S.O,Indian Restaurant,Smoke Shop,Bus Station,Farmers Market,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market
35,Chembur H.O,Indian Restaurant,Pizza Place,Gym,Snack Place,Garden,Fast Food Restaurant,Bakery,Dessert Shop,Deli / Bodega,Cupcake Shop
39,Ghatkopar West S.O,Indian Restaurant,Bakery,Juice Bar,Coffee Shop,Accessories Store,Men's Store,Vegetarian / Vegan Restaurant,Gym / Fitness Center,Bed & Breakfast,Donut Shop
40,Govandi S.O,Shop & Service,Indian Restaurant,Zoo,Farm,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market,Field
42,Kurla North S.O,Indian Restaurant,Hotel,Department Store,Mughlai Restaurant,Chinese Restaurant,Deli / Bodega,Fast Food Restaurant,Food & Drink Shop,Food,Flower Shop
48,Powai Iit S.O,Indian Restaurant,Dance Studio,Bar,Zoo,Farmers Market,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market
65,Malad East S.O,Indian Restaurant,Zoo,Falafel Restaurant,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market,Field,Fast Food Restaurant


In [37]:
# Check the neighborhood in Cluster 1 of Mumbai
mumbai_merged.loc[mumbai_merged['Cluster Labels'] == 1, mumbai_merged.columns[[4] + list(range(6, mumbai_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,BEST STaff Quarters S.O,Indian Restaurant,Gym / Fitness Center,Bar,Park,Multicuisine Indian Restaurant,Pharmacy,Coffee Shop,Playground,Recording Studio,Restaurant
3,Chinchbunder H.O,Mini Golf,Indian Restaurant,Harbor / Marina,Furniture / Home Store,Café,Brewery,Fast Food Restaurant,Food Court,Food & Drink Shop,Food
4,Cotton Exchange S.O,Pizza Place,Hookah Bar,Furniture / Home Store,Arts & Crafts Store,Falafel Restaurant,Food,Flower Shop,Flea Market,Fish Market,Field
5,Dadar Colony S.O,Vegetarian / Vegan Restaurant,Coffee Shop,Fast Food Restaurant,Juice Bar,Farmers Market,Shopping Mall,Indian Restaurant,Café,Maharashtrian Restaurant,Lounge
7,Kidwai Nagar S.O (Mumbai),Pizza Place,Garden,Coffee Shop,Café,Farm,Food,Flower Shop,Flea Market,Fish Market,Field
...,...,...,...,...,...,...,...,...,...,...,...
102,Jekegram S.O,Soccer Field,Indian Restaurant,Café,Lake,Food Court,Deli / Bodega,Food & Drink Shop,Food,Flower Shop,Flea Market
104,Kasarvadavali S.O,Shopping Mall,Pizza Place,Sporting Goods Shop,Coffee Shop,Clothing Store,Fast Food Restaurant,Food Court,Gym,Flower Shop,Fish Market
105,Kopri Colony S.O,Tennis Court,Dessert Shop,Athletics & Sports,Food Truck,Farm,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market
106,Thane Bazar S.O,Pizza Place,Dessert Shop,Diner,Chinese Restaurant,Fast Food Restaurant,Farm,Food & Drink Shop,Food,Flower Shop,Flea Market


In [38]:
# Check the neighborhood in Cluster 2 of Mumbai
mumbai_merged.loc[mumbai_merged['Cluster Labels'] == 2, mumbai_merged.columns[[4] + list(range(6, mumbai_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
88,Airoli B.O,Business Service,Zoo,Farmers Market,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market,Field


In [39]:
# Check the neighborhood in Cluster 3 of Mumbai
mumbai_merged.loc[mumbai_merged['Cluster Labels'] == 3, mumbai_merged.columns[[4] + list(range(6, mumbai_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
33,Bhandup Ind. Estate S.O,ATM,Snack Place,Cupcake Shop,Farmers Market,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market
90,Bokadvira B.O,ATM,Farm,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market,Field,Fast Food Restaurant
100,Dahisar B.O,ATM,Farm,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market,Field,Fast Food Restaurant


In [40]:
# Check the neighborhood in Cluster 4 of Mumbai
mumbai_merged.loc[mumbai_merged['Cluster Labels'] == 4, mumbai_merged.columns[[4] + list(range(6, mumbai_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,Anushakti Nagar S.O,Indian Restaurant,Fast Food Restaurant,Zoo,Falafel Restaurant,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market,Field
61,Jogeshwari East S.O,Fast Food Restaurant,Pedestrian Plaza,Pharmacy,Zoo,Farm,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market
72,Kalbadevi H.O,Indian Restaurant,Snack Place,Train Station,Fast Food Restaurant,Café,Farm,Food & Drink Shop,Food,Flower Shop,Flea Market
89,Belapur Node-- III S.O,Fast Food Restaurant,Indian Restaurant,BBQ Joint,Bus Station,Train Station,Bed & Breakfast,Liquor Store,Farmers Market,Food & Drink Shop,Food
92,Ghansoli S.O,Train Station,Bus Station,Zoo,Farm,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market,Field
95,Kopar Khairne S.O,Fast Food Restaurant,Indian Restaurant,Thai Restaurant,Zoo,Falafel Restaurant,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market


In [41]:
# Check the neighborhood in Cluster 0 of Pune
pune_merged.loc[pune_merged['Cluster Labels'] == 0, pune_merged.columns[[4] + list(range(6, pune_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Ammunition Factory Khadki S.O,Chinese Restaurant,Indian Restaurant,Wine Shop,English Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Distillery,Donut Shop
8,Bibvewadi S.O,Indian Restaurant,Vegetarian / Vegan Restaurant,Mobile Phone Shop,English Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Distillery,Donut Shop
13,Chinchwadgaon S.O,Indian Restaurant,Boat or Ferry,Falafel Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Distillery,Donut Shop,Dosa Place
20,Kasarwadi S.O,ATM,Fast Food Restaurant,Indian Restaurant,English Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Distillery,Donut Shop
21,Khondhwa KH B.O,Indian Restaurant,Middle Eastern Restaurant,Multiplex,Vegetarian / Vegan Restaurant,BBQ Joint,Gym,Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant
42,Dhankawadi S.O,Indian Restaurant,Café,Sandwich Place,Snack Place,Fast Food Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Distillery
43,Dhayari B.O,Indian Restaurant,Wine Shop,English Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Distillery,Donut Shop,Dosa Place
45,Karvenagar S.O,Café,Indian Restaurant,Motorcycle Shop,Wine Shop,Electronics Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Distillery
47,Khadakwasla R.S. S.O,Indian Restaurant,Wine Shop,English Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Distillery,Donut Shop,Dosa Place


In [42]:
# Check the neighborhood in Cluster 1 of Pune
pune_merged.loc[pune_merged['Cluster Labels'] == 1, pune_merged.columns[[4] + list(range(6, pune_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,9 DRD B.O,Café,Fast Food Restaurant,Hotel,Pizza Place,Indian Restaurant,Italian Restaurant,Diner,Sandwich Place,Hookah Bar,Coffee Shop
1,AFMC S.O,Café,Italian Restaurant,Wine Shop,Falafel Restaurant,Coffee Shop,Liquor Store,Chinese Restaurant,Other Great Outdoors,Diner,Donut Shop
7,Bhosarigoan S.O,Mobile Phone Shop,Pharmacy,Hotel,Karnataka Restaurant,Wine Shop,Dumpling Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
9,C D A (O) S.O,Hotel,Platform,Fast Food Restaurant,Clothing Store,Movie Theater,Indian Restaurant,Chinese Restaurant,Restaurant,Coffee Shop,Dessert Shop
12,Chinchwad East S.O,Indian Restaurant,Restaurant,Arts & Crafts Store,Gym,Diner,Playground,Department Store,Dessert Shop,Dim Sum Restaurant,Distillery
14,Dapodi Bazar S.O,Coffee Shop,Department Store,Dessert Shop,Snack Place,Fast Food Restaurant,Electronics Store,Deli / Bodega,Dim Sum Restaurant,Diner,Distillery
16,Gondhale Nagar S.O,Snack Place,Indian Restaurant,Pizza Place,Fast Food Restaurant,Ice Cream Shop,Convenience Store,Dumpling Restaurant,Deli / Bodega,Department Store,Dessert Shop
17,Hadpsar I.E. S.O,Indian Restaurant,Fast Food Restaurant,Garden,Gym,Coffee Shop,Chinese Restaurant,Juice Bar,Plaza,Ice Cream Shop,Bakery
18,Infotech Park (Hinjawadi) S.O,Hotel,Multiplex,Department Store,Italian Restaurant,Indian Restaurant,Food Court,Food Truck,Food & Drink Shop,Dosa Place,Deli / Bodega
19,Kalewadi B.O,ATM,Gym / Fitness Center,Men's Store,Indian Restaurant,Food Truck,Dumpling Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Furniture / Home Store


In [43]:
# Check the neighborhood in Cluster 2 of Pune
pune_merged.loc[pune_merged['Cluster Labels'] == 2, pune_merged.columns[[4] + list(range(6, pune_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Airport S.O (Pune),Recreation Center,Lake,Wine Shop,Dumpling Restaurant,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Distillery
48,Kudje B.O,Recreation Center,Wine Shop,Electronics Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Distillery,Donut Shop


In [44]:
# Check the neighborhood in Cluster 3 of Pune
pune_merged.loc[pune_merged['Cluster Labels'] == 3, pune_merged.columns[[4] + list(range(6, pune_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Bhosari I.E. S.O,Shoe Store,Wine Shop,English Restaurant,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Distillery,Donut Shop


In [45]:
# Check the neighborhood in Cluster 4 of Pune
pune_merged.loc[pune_merged['Cluster Labels'] == 4, pune_merged.columns[[4] + list(range(6, pune_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Aundh Camp S.O,Café,Wine Shop,Dance Studio,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Distillery,Donut Shop,Dosa Place


In [46]:
# Check the neighborhood in Cluster 0 of Nagpur
nagpur_merged.loc[nagpur_merged['Cluster Labels'] == 0, nagpur_merged.columns[[4] + list(range(6, nagpur_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Mhalginagar S.O,Mobile Phone Shop,IT Services,Wings Joint,Garden,Food Truck,Food Court,Fast Food Restaurant,Dumpling Restaurant,Dhaba,Dessert Shop
27,Adegaon B.O,Mobile Phone Shop,Wings Joint,Gift Shop,Garden,Food Truck,Food Court,Fast Food Restaurant,Dumpling Restaurant,Dhaba,Dessert Shop


In [47]:
# Check the neighborhood in Cluster 1 of Nagpur
nagpur_merged.loc[nagpur_merged['Cluster Labels'] == 1, nagpur_merged.columns[[4] + list(range(6, nagpur_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,Mahal S.O (Nagpur),Indian Restaurant,Wings Joint,Department Store,Gift Shop,Garden,Food Truck,Food Court,Fast Food Restaurant,Dumpling Restaurant,Dhaba


In [48]:
# Check the neighborhood in Cluster 2 of Nagpur
nagpur_merged.loc[nagpur_merged['Cluster Labels'] == 2, nagpur_merged.columns[[4] + list(range(6, nagpur_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Abhyankar Nagar S.O,Gift Shop,Bakery,Gym / Fitness Center,Shopping Mall,Hotel,Park,Outdoors & Recreation,Indian Restaurant,Dessert Shop,Food Truck
1,Ajni S.O,Pool,IT Services,Business Service,Chinese Restaurant,Wings Joint,Department Store,Garden,Food Truck,Food Court,Fast Food Restaurant
2,Ayodhya Nagar S.O,Garden,Food Court,Wings Joint,Gym / Fitness Center,Gift Shop,Food Truck,Fast Food Restaurant,Dumpling Restaurant,Dhaba,Dessert Shop
3,Bagadganj S.O,ATM,Clothing Store,Department Store,Gift Shop,Garden,Food Truck,Food Court,Fast Food Restaurant,Dumpling Restaurant,Dhaba
5,Bezonbagh S.O,Mobile Phone Shop,Bakery,History Museum,Playground,Coffee Shop,Garden,Food Truck,Food Court,Fast Food Restaurant,Dumpling Restaurant
6,Borgaon Road S.O,Bakery,Pizza Place,Café,Coffee Shop,Department Store,Wings Joint,Dessert Shop,Gift Shop,Garden,Food Truck
7,Bureau Of Mines S.O,Sandwich Place,Restaurant,Pizza Place,Burger Joint,Indian Restaurant,Wings Joint,Coffee Shop,Food Truck,Food Court,Fast Food Restaurant
9,Congress Nagar S.O (Nagpur),Shopping Mall,Lounge,Gym,Restaurant,Bar,Fast Food Restaurant,Hotel,Indian Restaurant,Italian Restaurant,Department Store
14,Khamla S.O,Coffee Shop,Food Truck,Food Court,Pizza Place,Wings Joint,Gift Shop,Garden,Fast Food Restaurant,Dumpling Restaurant,Dhaba
15,Laxmi Nagar S.O (Nagpur),Wings Joint,Pizza Place,Department Store,Clothing Store,Bar,Dessert Shop,Gift Shop,Garden,Food Truck,Food Court


In [49]:
# Check the neighborhood in Cluster 3 of Nagpur
nagpur_merged.loc[nagpur_merged['Cluster Labels'] == 3, nagpur_merged.columns[[4] + list(range(6, nagpur_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Jaitala S.O,Dhaba,Wings Joint,Gym / Fitness Center,Gift Shop,Garden,Food Truck,Food Court,Fast Food Restaurant,Dumpling Restaurant,Dessert Shop


In [50]:
# Check the neighborhood in Cluster 4 of Nagpur
nagpur_merged.loc[nagpur_merged['Cluster Labels'] == 4, nagpur_merged.columns[[4] + list(range(6, nagpur_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Ganjipeth S.O,Pharmacy,Wings Joint,Coffee Shop,Gift Shop,Garden,Food Truck,Food Court,Fast Food Restaurant,Dumpling Restaurant,Dhaba
